In [1]:
#from attractions_recc import *
from IPython.display import display
import ipywidgets as w
import pandas as pd
import re

In [2]:
from attractions_recc import *

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
att_df = pd.read_json('etl/attractions.json',orient='records')

## User details and Preferences

In [4]:
'''uname = w.Text(description="User Name")
place = w.Text(value = 'Province',  description="Destination")
budget = w.IntRangeSlider(min=att_df.price.min(), max=att_df.price.max(),step=10,value=[att_df.price.min(),att_df.price.max()],description="Budget")
v1 = w.VBox([uname,place,budget])
start = w.DatePicker(description='Start Date',disabled=False)
end = w.DatePicker(description='End Date',disabled=False)
v2 = w.VBox([start,end])
out = w.HBox([v1,v2])
display(out)'''
uname = 'Sagar Patl'
place = 'Ontario'
budget = (5,900)
start = '2022-01-25'
end = '2022-01-28'

In [5]:
print(uname)
print(place)
print(start)
print(end)
print(budget)
#print(budget.max)

Sagar Patl
Ontario
2022-01-25
2022-01-28
(5, 900)


In [6]:
'''category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:20]
categories = list(category_df.category.values)
cat_rat = dict()
def on_button_clicked(b):
    if b.description in cat_rat:
        return
    print(b.description)
    slider = w.IntSlider(min=0,max=5,step=1,description='Rate')
    display(slider)
    cat_rat[b.description] = slider
    if(len(cat_rat) < 5):
        print("Rate {x} more!\n".format(x=5-len(cat_rat)))
    
but_layout = w.Layout(width='100%', height='100px')
but_items = [w.Button(description=c, layout=but_layout) for c in categories]
on_clk = [item.on_click(on_button_clicked) for item in but_items]
c1 = w.VBox([i for i in but_items[:4]])
c2 = w.VBox([i for i in but_items[4:8]])
c3 = w.VBox([i for i in but_items[8:12]])
c4 = w.VBox([i for i in but_items[12:16]])
c5 = w.VBox([i for i in but_items[16:]])
buttons = w.HBox([c1,c2,c3,c4,c5])
print("Select and rate atleast 5 categories and rate them:")
display(buttons)'''

category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:20]
categories = list(category_df.category.values)
cat_rat = dict()
cat_rat = {
    'private_&_custom_tours': 2, 
    'food,_wine_&_nightlife': 4, 
    'tours_&_sightseeing': 5, 
    'family_friendly': 3, 
    'day_trips_&_excursions': 2
}

In [7]:
print(categories)
print(cat_rat)

['private_&_custom_tours', 'tours_&_sightseeing', 'recommended_experiences', 'outdoor_activities', 'food,_wine_&_nightlife', 'family_friendly', 'cultural_&_theme_tours', 'walking_&_biking_tours', 'day_trips_&_excursions', 'water_sports', 'cruises,_sailing_&_water_tours', 'transfers_&_ground_transport', 'luxury_&_special_occasions', 'sightseeing_tickets_&_passes', 'multi-day_&_extended_tours', 'shore_excursions', 'air,_helicopter_&_balloon_tours', 'holiday_&_seasonal_tours', 'shows,_concerts_&_sports', 'classes_&_workshops']
{'private_&_custom_tours': 2, 'food,_wine_&_nightlife': 4, 'tours_&_sightseeing': 5, 'family_friendly': 3, 'day_trips_&_excursions': 2}


In [8]:
user_name = re.sub(' ','_',uname.lower())
province = re.sub(' ','_',place.lower())
(low,high) = tuple([float(i) for i in budget])
begin_date = start
end_date = end
cat_rating = dict()
for key, value in cat_rat.items():
    cat_rating[key] = float(value)

In [9]:
print(user_name)
print(province)
print(begin_date)
print(end_date)
print(cat_rating)

sagar_patl
ontario
2022-01-25
2022-01-28
{'private_&_custom_tours': 2.0, 'food,_wine_&_nightlife': 4.0, 'tours_&_sightseeing': 5.0, 'family_friendly': 3.0, 'day_trips_&_excursions': 2.0}


## Model recommendation and filtering

In [10]:
filename, user, rbm_att = get_recc(att_df, cat_rating)

with_url = filter_df(filename, user, low, high, province, att_df)

Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Similar User: 248
Model restored


In [11]:
print(filename)
print(user)
print(rbm_att)
print(with_url)

e50_r40000_lr0.01_hu128_bs16
248
       user_id  attraction_id  rbm_att_id  rating
0            0             74          48       5
1           20             74          48       5
2           34             74          48       5
3           57             74          48       5
4           96             74          48       5
...        ...            ...         ...     ...
33920    12932             59          41       5
33921    13005           1821         841       5
33922    13038           1598         723       1
33923    13071           3050        1279       5
33924    13092           3230        1300       5

[33925 rows x 4 columns]
                                                   name  \
5     niagara_falls_in_one_day:_deluxe_sightseeing_t...   
1310  best_of_niagara_falls_tour_from_niagara_falls,...   
2258                      niagara_falls_helicopter_tour   
3285                niagara_falls_day_tour_from_toronto   
2247  niagara_falls_day_and_evening_tour_with_

In [12]:
#%%capture
final = dict()
final['timeofday'] = []
final['image'] = []
final['name'] = []
final['location'] = []
final['price'] = []
final['rating'] = []
final['category'] = []

for i in range(1,4):
    for j in range(2):
        final['timeofday'].append('Morning')
    for j in range(2):
        final['timeofday'].append('Evening')

for i in range(len(final['timeofday'])): 
    if i%4 == 0: 
        final = top_recc(with_url, final)
    else:
        final = find_closest(with_url, final['location'][-1],final['timeofday'][i], final)


Item no.: 1 --> Item name = niagara_falls_in_one_day:_deluxe_sightseeing_tour_of_american_and_canadian_sides
Evaluating...

Item no.: 1 --> Item name = niagara_falls_canadian_side_tour_and_maid_of_the_mist_boat_ride
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = 6-hour_niagara_falls_canadian_side_evening_illuminations_tour
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = epic_toronto_night_tour
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = best_of_niagara_falls_tour_from_niagara_falls
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = ultimate_toronto_tour
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = niagara_falls_night_tour_with_dinner_and_cruise
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = day_and_night_tour_of_niagara_falls
Evaluating...
Starting Download...

Errors: 0


Item no.: 1 --> Item name = niagara_falls

In [13]:
#print(final)
import json
json_object = json.dumps(final, indent = 4) 
print(json_object)

#with open("sample_final.json", "w") as outfile:
    #json.dump(final, outfile)

{
    "timeofday": [
        "Morning",
        "Morning",
        "Evening",
        "Evening",
        "Morning",
        "Morning",
        "Evening",
        "Evening",
        "Morning",
        "Morning",
        "Evening",
        "Evening"
    ],
    "image": [
        "downloads\\noimage.jpg",
        "downloads/niagara_falls_canadian_side_tour_and_maid_of_the_mist_boat_ride\\1. niagara-falls-canadian.jpg",
        "downloads/6-hour_niagara_falls_canadian_side_evening_illuminations_tour\\1. 6-hour-niagara-falls.jpg",
        "downloads/epic_toronto_night_tour\\1. epic-toronto-night-tour.jpg",
        "downloads/best_of_niagara_falls_tour_from_niagara_falls\\1. best-of-niagara-falls-tour-from-niagara-falls-ontario-in-niagara-falls-190431.jpg",
        "downloads/ultimate_toronto_tour\\1. ultimate-toronto-tour.jpg",
        "downloads/niagara_falls_night_tour_with_dinner_and_cruise\\1. niagara-falls-night-tour-with-dinner-and-cruise-in-niagara-falls-218806.jpg",
        "downloa

## Final Output

In [24]:
days = 2 + 1
display(final_output(days,final))

[VBox(children=(HBox(children=(VBox(children=(HTML(value="<b><font color='orange'>MORNING</b>"), HTML(value="<b><font color='purple'>Recommendation 1:</b>"), Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x05\x03\x04\x04\x04\...', format='jpg', height='400', width='300'), HTML(value="<b><font color='black'>Niagara falls in one day: deluxe sightseeing tour of american and canadian sides</b>", description='NAME'), HTML(value="<b><font color='black'>Featured tours and tickets</b>", description='CATEGORY'), HTML(value="<b><font color='black'>(43.1024360657,-78.9616394043)</b>", description='LOCATION'), HTML(value="<b><font color='black'>204.42</b>", description='PRICE'), HTML(value="<b><font color='black'>5.0</b>", description='RATING')), layout=Layout(display='flex', flex_flow='column', justify_content='space-between', width='75%')), VBox(children=(HTML(value="<b><font color='orange'>EVENING</b>"), HTML(value="<b><font color='purple'>Recommen

In [15]:
final_output(days,final)

[VBox(children=(HBox(children=(VBox(children=(HTML(value="<b><font color='orange'>MORNING</b>"), HTML(value="<b><font color='purple'>Recommendation 1:</b>"), Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xdb\x00C\x00\x05\x03\x04\x04\x04\...', format='jpg', height='400', width='300'), HTML(value="<b><font color='black'>Niagara falls in one day: deluxe sightseeing tour of american and canadian sides</b>", description='NAME'), HTML(value="<b><font color='black'>Featured tours and tickets</b>", description='CATEGORY'), HTML(value="<b><font color='black'>(43.1024360657,-78.9616394043)</b>", description='LOCATION'), HTML(value="<b><font color='black'>204.42</b>", description='PRICE'), HTML(value="<b><font color='black'>5.0</b>", description='RATING')), layout=Layout(display='flex', flex_flow='column', justify_content='space-between', width='75%')), VBox(children=(HTML(value="<b><font color='orange'>EVENING</b>"), HTML(value="<b><font color='purple'>Recommen